In [105]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib2
import json
import argparse
import logging
import os
import io
import requests
import pandas as pd
import key # import api key

# make plots show up inline
%matplotlib inline 

In [11]:
api_key = key.API_KEY
base_url = 'http://www.giantbomb.com/api'

In [60]:
offset=0
payload = {'api_key': api_key, 'format':'json', 'offset':offset, 'limit':1}

In [110]:
offsets = [0, 101, 201, 301, 401, 501, 601]
gb = []
for x in offsets:
    payload = {'api_key': api_key, 'format':'json', 'offset':x}
    results = requests.get(base_url + "/reviews/", params=payload).json()['results']
    gb.append(results)
    # because giantbomb limits us to 100 records at a time,
    # save each record as a separate file
    with io.open('%s.json' % x, 'w', encoding='utf-8') as f:
        f.write(unicode(json.dumps(results, ensure_ascii=False)))